#### Creacion de funciones para api

In [1]:
def generate_user_preferences(res_w, res_sc, rel_w, rel_sc, rec_w, rec_sc, ct):
    # Definir los pesos y subcategorías
    weights = [0, 1, 2, 3]
    restaurant_subcategories = [1, 2, 3, 4, 5]
    religion_subcategories = [1, 2, 3, 4, 5]
    recreation_subcategories = [1, 2, 3]
    chronotypes = [0, 1]  # Incluye el 2 para "Indiferente/ambos"

    # Validar las entradas
    if res_w not in weights:
        raise ValueError(f"res_w debe estar en {weights}, pero se recibió {res_w}")
    if res_sc not in restaurant_subcategories:
        raise ValueError(f"res_sc debe estar en {restaurant_subcategories}, pero se recibió {res_sc}")
    if rel_w not in weights:
        raise ValueError(f"rel_w debe estar en {weights}, pero se recibió {rel_w}")
    if rel_sc not in religion_subcategories:
        raise ValueError(f"rel_sc debe estar en {religion_subcategories}, pero se recibió {rel_sc}")
    if rec_w not in weights:
        raise ValueError(f"rec_w debe estar en {weights}, pero se recibió {rec_w}")
    if rec_sc not in recreation_subcategories:
        raise ValueError(f"rec_sc debe estar en {recreation_subcategories}, pero se recibió {rec_sc}")
    if ct not in chronotypes:
        raise ValueError(f"ct debe estar en {chronotypes}, pero se recibió {ct}")

    # Generar preferencias del usuario
    user_preferences = {
        "Restaurant": {
            "weight": res_w,
            "sub_category": res_sc
        },
        "Religion": {
            "weight": rel_w,
            "sub_category": rel_sc
        },
        "Recreation": {
            "weight": rec_w,
            "sub_category": rec_sc
        },
        "Cronotipo": ct
    }
    
    return user_preferences

In [2]:
def group_coordinates(*coords):
    lat_inm = []
    lon_inm = []

    for i in range(0, len(coords), 2):
        lat_inm.append(coords[i])
        lon_inm.append(coords[i+1])

    return lat_inm, lon_inm

# # Ejemplo de uso
# lat_1 = 34.058092
# lon_1 = -118.292130
# lat_2 = 34.032025
# lon_2 = -118.244198
# lat_3 = 34.047421
# lon_3 = -118.256777
# lat_4 = 34.027630
# lon_4 = -118.266843
# lat_5 = 34.015112
# lon_5 = -118.251910
# lat_6 = 33.993401
# lon_6 = -118.278310

# lat_inm, lon_inm = group_coordinates(
#     lat_1, lon_1,
#     lat_2, lon_2,
#     lat_3, lon_3,
#     lat_4, lon_4,
#     lat_5, lon_5,
#     lat_6, lon_6
# )

In [3]:
# calcular_distancia: Función accesoria que se utiliza para calcular la distancia lineal entre dos puntos

def calcular_distancia(lat_1, long_1, lat_2, long_2):
    # Constantes para convertir grados a metros
    METROS_POR_GRADO_LAT = 111320  # Aprox. para todas las latitudes
    METROS_POR_GRADO_LONG = 111320 * math.cos(math.radians(lat_1))  # Varía según la latitud

    # Diferencias en grados
    delta_lat = lat_2 - lat_1
    delta_long = long_2 - long_1

    # Convertir diferencias en grados a diferencias en metros
    delta_lat_metros = delta_lat * METROS_POR_GRADO_LAT
    delta_long_metros = delta_long * METROS_POR_GRADO_LONG

    # Calcular distancia euclidiana
    distancia_metros = math.sqrt(delta_lat_metros**2 + delta_long_metros**2)
    
    # Redondear a la centena más cercana
    distancia_redondeada = round(distancia_metros, -2)
    
    return distancia_redondeada

In [4]:
# filtrar_locales: Define los locales a tener en cuenta a partir de la vecindad del inmueble y las preferencias del usuario

def filtrar_locales(df_locales, lat_inm, lon_inm, km, user_pref):
    
    df_locales_copy = df_locales.copy()

    # Función para determinar la categoría
    def determinar_categoria(row):
        if any(row[['res_asian', 'res_latin', 'res_euro', 'res_fast', 'res_vegan']] != 0):
            return 'res'
        elif row['religion'] != 0:
            return 'rel'
        elif row['recreation'] != 0:
            return 'rec'
        elif row['bienestar'] != 0:
            return 'bien'
        else:
            return np.nan

    # Crear la nueva columna 'ml_category'
    df_locales_copy['ml_category'] = df_locales_copy.apply(determinar_categoria, axis=1)
    
    # Filtrar el DataFrame según el área determinada
    df_locales_copy = df_locales_copy[
    (df_locales_copy['latitude'] >= (lat_inm - 0.0045 * km * 2)) & 
    (df_locales_copy['latitude'] <= (lat_inm + 0.0045 * km * 2)) &
    (df_locales_copy['longitude'] >= (lon_inm - 0.0054 * km * 2)) & 
    (df_locales_copy['longitude'] <= (lon_inm + 0.0054 * km * 2))
    ]
    
    # Filtrar el DataFrame a partir de las preferencias del usuario

    # Filtrar según el peso
    if user_pref['Restaurant']['weight'] == 0:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_asian'] != 1) &
            (df_locales_copy['res_latin'] != 1) &
            (df_locales_copy['res_euro'] != 1) &
            (df_locales_copy['res_fast'] != 1) &
            (df_locales_copy['res_vegan'] != 1)
        ]
    if user_pref['Religion']['weight'] == 0:
        df_locales_copy = df_locales_copy[df_locales_copy['religion'] != 1]
    if user_pref['Recreation']['weight'] == 0:
        df_locales_copy = df_locales_copy[df_locales_copy['recreation'] != 1]

    # Filtrar según la sub-categoría para "Restaurant"
    restaurant_sub_category = user_pref['Restaurant']['sub_category']
    if restaurant_sub_category == 1:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_asian'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 2:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_latin'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 3:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_euro'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 4:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_fast'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]
    elif restaurant_sub_category == 5:
        df_locales_copy = df_locales_copy[
            (df_locales_copy['res_vegan'] == 1) |
            ((df_locales_copy['res_asian'] == 0) & 
            (df_locales_copy['res_latin'] == 0) & 
            (df_locales_copy['res_euro'] == 0) & 
            (df_locales_copy['res_fast'] == 0) & 
            (df_locales_copy['res_vegan'] == 0))
        ]

    # Filtrar según la sub-categoría para "Religion"
    religion_sub_category = user_pref['Religion']['sub_category']
    df_locales_copy = df_locales_copy[
        (df_locales_copy['religion'] == religion_sub_category) |
        (df_locales_copy['religion'] == 0)
    ]

    # Filtrar según la sub-categoría para "Recreation"
    recreation_sub_category = user_pref['Recreation']['sub_category']
    df_locales_copy = df_locales_copy[
        (df_locales_copy['recreation'] == recreation_sub_category) |
        (df_locales_copy['recreation'] == 0)
    ]

    # Dejo solo columnas de interes
    
    # Añadir columna "distance" al DataFrame
    df_locales_copy['distance'] = df_locales_copy.apply(
        lambda row: calcular_distancia(row['latitude'], row['longitude'], lat_inm, lon_inm),
        axis=1
    )
    df_filtrado = df_locales_copy[['address','gmap_id', 'latitude', 'longitude','avg_rating',
       'num_of_reviews','religion', 'recreation', 'bienestar', 'avg_rating_correction','forecast', 'tendencia','rating_3_months', 'time_category', 'distance','ml_category']]

    return df_filtrado

In [5]:
# calcular_puntuacion: Se ingresa el df filtrado por la función anterior y las preferencias de usuario. Devuelve la puntuación del inmueble dado y cadenas de texto
# con la distancia y puntuación de los mejores locales de cada categoría

def calcular_puntuacion(df_prueba, user_pref):
    total_score = 0
    score_res = 0
    score_rel = 0
    score_rec = 0
    score_bien = 0

    best_restaurant = ""
    best_bienestar = ""
    best_recreation = ""
    best_religion = ""

    df = df_prueba.copy()

    # Restaurantes (bienestar, recreation, religion son 0)
    restaurants = df[(df['bienestar'] == 0) & (df['recreation'] == 0) & (df['religion'] == 0)]
    if len(restaurants) > 0:
        if len(restaurants) == 1:
            score_res = 4
        elif len(restaurants) in [2, 3]:
            score_res = 4.3
        else:
            score_res = 4.5
        
        max_avg_rating = max(restaurants['avg_rating'].max(), restaurants['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_res += 0.3
        
        # Encontrar el restaurante con el max_avg_rating más alto
        best_restaurant_idx = restaurants.loc[
            (restaurants['avg_rating'] == max_avg_rating) |
            (restaurants['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_restaurant_idx, 'forecast'] == 'fuerte':
            if df.at[best_restaurant_idx, 'tendencia'] == 'Alta':
                score_res += 0.2
        elif df.at[best_restaurant_idx, 'tendencia'] == 'Alta':
            score_res += 0.1

        # Guardar el mejor restaurante
        best_restaurant = df.at[best_restaurant_idx, 'address']
        best_restaurant_name = best_restaurant.split(',')[0]
        best_restaurant_rating = max(
        df.at[best_restaurant_idx, 'avg_rating'], 
        df.at[best_restaurant_idx, 'avg_rating_correction'] if pd.notna(df.at[best_restaurant_idx, 'avg_rating_correction']) else -np.inf
        )
        best_restaurant_rating = round(best_restaurant_rating, 2)
        best_restaurant_dist = df.at[best_restaurant_idx, 'distance']
        best_restaurant_fc = df.at[best_restaurant_idx, 'rating_3_months']

        
    
    # Bienestar (bienestar es 1)
    bienestar = df[df['bienestar'] == 1]
    if len(bienestar) > 0:
        if len(bienestar) == 1:
            score_bien = 1.5
        elif len(bienestar) in [2, 3]:
            score_bien = 2
        else:
            score_bien = 2.5
        
        max_avg_rating = max(bienestar['avg_rating'].max(), bienestar['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_bien += 0.3
        
        best_bienestar_idx = bienestar.loc[
            (bienestar['avg_rating'] == max_avg_rating) |
            (bienestar['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_bienestar_idx, 'forecast'] == 'fuerte':
            if df.at[best_bienestar_idx, 'tendencia'] == 'Alta':
                score_bien += 0.2
        elif df.at[best_bienestar_idx, 'tendencia'] == 'Alta':
            score_bien += 0.1
        
        # Guardar el mejor bienestar
        best_bienestar = df.at[best_bienestar_idx, 'address']
        best_bienestar_name = best_bienestar.split(',')[0]
        best_bienestar_rating = max(
        df.at[best_bienestar_idx, 'avg_rating'], 
        df.at[best_bienestar_idx, 'avg_rating_correction'] if pd.notna(df.at[best_bienestar_idx, 'avg_rating_correction']) else -np.inf
        )
        best_bienestar_rating = round(best_bienestar_rating, 2)
        best_bienestar_dist = df.at[best_bienestar_idx, 'distance']
        best_bienestar_fc = df.at[best_bienestar_idx, 'rating_3_months']
    
    # Recreación (recreation es distinto de 0)
    recreation = df[df['recreation'] != 0]
    if len(recreation) > 0:
        if len(recreation) == 1:
            score_rec = 0.5
        elif len(recreation) in [2, 3]:
            score_rec = 0.6
        else:
            score_rec = 0.7
        
        max_avg_rating = max(recreation['avg_rating'].max(), recreation['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_rec += 0.1
        
        best_recreation_idx = recreation.loc[
            (recreation['avg_rating'] == max_avg_rating) |
            (recreation['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_recreation_idx, 'forecast'] == 'fuerte':
            if df.at[best_recreation_idx, 'tendencia'] == 'Alta':
                score_rec += 0.2
        elif df.at[best_recreation_idx, 'tendencia'] == 'Alta':
            score_rec += 0.1
        
        # Guardar el mejor recreación
        best_recreation = df.at[best_recreation_idx, 'address']
        best_recreation_name = best_recreation.split(',')[0]
        best_recreation_rating = max(
        df.at[best_recreation_idx, 'avg_rating'], 
        df.at[best_recreation_idx, 'avg_rating_correction'] if pd.notna(df.at[best_recreation_idx, 'avg_rating_correction']) else -np.inf
        )
        best_recreation_rating = round(best_recreation_rating, 2)
        best_recreation_dist = df.at[best_recreation_idx, 'distance']
        best_recreation_fc = df.at[best_recreation_idx, 'rating_3_months']
    
    # Religión (religion es distinto de 0)
    religion = df[df['religion'] != 0]
    if len(religion) > 0:
        if len(religion) == 1:
            score_rel = 0.5
        elif len(religion) in [2, 3]:
            score_rel = 0.6
        else:
            score_rel = 0.7
        
        max_avg_rating = max(religion['avg_rating'].max(), religion['avg_rating_correction'].max())
        if max_avg_rating > 4.5:
            score_rel += 0.1
        
        best_religion_idx = religion.loc[
            (religion['avg_rating'] == max_avg_rating) |
            (religion['avg_rating_correction'] == max_avg_rating)
        ].index[0]
        
        if df.at[best_religion_idx, 'forecast'] == 'fuerte':
            if df.at[best_religion_idx, 'tendencia'] == 'Alta':
                score_rel += 0.2
        elif df.at[best_religion_idx, 'tendencia'] == 'Alta':
            score_rel += 0.1
        
        # Guardar el mejor religión
        best_religion = df.at[best_religion_idx, 'address']
        best_religion_name = best_religion.split(',')[0]
        best_religion_rating = max(
        df.at[best_religion_idx, 'avg_rating'], 
        df.at[best_religion_idx, 'avg_rating_correction'] if pd.notna(df.at[best_religion_idx, 'avg_rating_correction']) else -np.inf
        )
        best_religion_rating = round(best_religion_rating, 2)
        best_religion_dist = df.at[best_religion_idx, 'distance']
        best_religion_fc = df.at[best_religion_idx, 'rating_3_months']


    # Ponderación final del puntaje:
    p_res = user_pref['Restaurant']['weight'] # Puede ser 0, 1, 2 o 3
    p_rec = user_pref['Recreation']['weight']
    p_rel = user_pref['Religion']['weight']

    def ajustar_score(score, ponderacion):
        if ponderacion == 1:
            return score * 0.8
        elif ponderacion == 3:
            return score + (5 - score) * 0.1
        return score

    score_res = ajustar_score(score_res, p_res)
    score_rec = ajustar_score(score_rec, p_rec)
    score_rel = ajustar_score(score_rel, p_rel)
    
    total_score = score_res + score_rel + score_bien + score_rec

    if p_res == 0:
        total_score = (5 * total_score) / 10
        
    elif p_rec == 0:
        total_score = (9 * total_score) / 10
    
    elif p_rel == 0:
        total_score = (9 * total_score) / 10
    
    messages = []

    def agregar_mensaje(category_name, category_dist, category_rating, category_fc, category_label):
        if category_fc is not None and not pd.isna(category_fc):
            return f"Tienes un {category_label} llamado '{category_name}' a {category_dist} metros con una valoración histórica de {category_rating} y una predicción de {category_fc} dentro de tres meses"
        else:
            return f"Tienes un {category_label} llamado '{category_name}' a {category_dist} metros con una valoración histórica de {category_rating}"

    if best_restaurant:
        messages.append(agregar_mensaje(best_restaurant_name, best_restaurant_dist, best_restaurant_rating, best_restaurant_fc, 'restaurante'))
    if best_bienestar:
        messages.append(agregar_mensaje(best_bienestar_name, best_bienestar_dist, best_bienestar_rating, best_bienestar_fc, best_bienestar_name))
    if best_recreation:
        messages.append(agregar_mensaje(best_recreation_name, best_recreation_dist, best_recreation_rating, best_recreation_fc, 'lugar de recreación'))
    if best_religion:
        messages.append(agregar_mensaje(best_religion_name, best_religion_dist, best_religion_rating, best_religion_fc, best_religion_name))
    return total_score, score_res, score_rel, score_rec, score_bien, messages

In [6]:
# def obtener_recomendacion(df_base_datos, user_pref, km, lat_inm, lon_inm):
#     # Crear una lista para almacenar los resultados
#     resultados = []

#     for lat, lon in zip(lat_inm, lon_inm):
#         df_bd_filtrada = filtrar_locales(df_base_datos, lat, lon, km, user_pref)
#         score_inm, score_res, score_rel, score_rec, score_bien, mensajes = calcular_puntuacion(df_bd_filtrada, user_pref)
        
#         # Agregar los resultados a la lista
#         resultados.append({
#             'score_inm': score_inm,
#             'lat': lat,
#             'lon': lon,
#             'mensajes': mensajes
#         })

#     # Convertir la lista de resultados a un DataFrame
#     resultados_df = pd.DataFrame(resultados)
    
#     return resultados_df

In [7]:

def obtener_recomendacion(df_base_datos, user_pref, km, lat_inm, lon_inm):
    # Crear listas para almacenar los resultados y los DataFrames filtrados
    resultados = []
    df_locales_list = []

    # Iterar sobre las coordenadas
    for i, (lat, lon) in enumerate(zip(lat_inm, lon_inm)):
        df_bd_filtrada = filtrar_locales(df_base_datos, lat, lon, km, user_pref)
        score_inm, score_res, score_rel, score_rec, score_bien, mensajes = calcular_puntuacion(df_bd_filtrada, user_pref)
        
        # Crear el código del inmueble
        inmueble_codigo = f'inm_{i + 1}'
        
        # Agregar los resultados a la lista
        resultados.append({
            'inmueble': inmueble_codigo,
            'score_inm': score_inm,
            'lat': lat,
            'lon': lon,
            'mensajes': mensajes
        })

        # Modificar el DataFrame filtrado para agregar las columnas necesarias
        df_bd_filtrada['inmueble'] = inmueble_codigo
        df_bd_filtrada['rating'] = df_bd_filtrada['avg_rating_correction'].fillna(df_bd_filtrada['avg_rating']).round(2)
        
        # Seleccionar y renombrar las columnas necesarias
        df_locales = df_bd_filtrada[['inmueble', 'address', 'latitude', 'longitude', 'rating', 'rating_3_months', 'time_category', 'ml_category']]
        
        # Agregar el DataFrame a la lista
        df_locales_list.append(df_locales)
    
    # Convertir la lista de resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados)
    
    # Concatenar los DataFrames de locales
    locales_df = pd.concat(df_locales_list, ignore_index=True)
    
    return resultados_df, locales_df

In [8]:
# Carga de librerías

import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.
import math

In [9]:
df_locales_ml = pd.read_csv("df_locales_ml.csv")


In [10]:
# Ejemplo de uso
lat_1 = 34.058092
lon_1 = -118.292130
lat_2 = 34.032025
lon_2 = -118.244198
lat_3 = 34.047421
lon_3 = -118.256777
lat_4 = 34.027630
lon_4 = -118.266843
lat_5 = 34.015112
lon_5 = -118.251910
lat_6 = 33.993401
lon_6 = -118.278310

lat_inm, lon_inm = group_coordinates(
     lat_1, lon_1,
     lat_2, lon_2,
     lat_3, lon_3,
     lat_4, lon_4,
     lat_5, lon_5,
     lat_6, lon_6
 )

user_pref = generate_user_preferences(3, 4, 2, 1, 1, 1, 0)
resultados,  locales = obtener_recomendacion(df_locales_ml, user_pref, 1, lat_inm, lon_inm)

AttributeError: 'float' object has no attribute 'round'

In [ ]:
resultados

,inmueble,score_inm,lat,lon,mensajes
0,inm_1,9.07,34.058092,-118.292130,[Tienes un restaurante llamado 'Fire Chicken' ...
1,inm_2,8.11,34.032025,-118.244198,[Tienes un restaurante llamado 'Es Todo Vegan ...
2,inm_3,8.86,34.047421,-118.256777,[Tienes un restaurante llamado 'Danny Boy's Fa...
3,inm_4,8.71,34.027630,-118.266843,[Tienes un restaurante llamado 'JT Cafe' a 900...
4,inm_5,6.90,34.015112,-118.251910,[Tienes un restaurante llamado 'Maximo's Pizza...
5,inm_6,7.46,33.993401,-118.278310,[Tienes un restaurante llamado 'Domino's Pizza...


In [ ]:
locales

,inmueble,latitude,longitude,rating,rating_3_months,time_category,ml_category
0,inm_1,34.056428,-118.291801,3.00,NaN,0.0,bien
1,inm_1,34.052670,-118.290634,3.50,NaN,0.0,bien
2,inm_1,34.054407,-118.296432,4.50,NaN,0.0,rel
3,inm_1,34.060011,-118.283239,3.00,NaN,0.0,bien
4,inm_1,34.061971,-118.295066,3.92,3.71,0.0,res
...,...,...,...,...,...,...,...
273,inm_6,33.992088,-118.286828,4.40,NaN,0.0,rel
274,inm_6,33.985354,-118.287453,4.20,NaN,0.0,rel
275,inm_6,33.992982,-118.283074,3.40,NaN,1.0,res
276,inm_6,33.988882,-118.273749,2.80,NaN,0.0,bien


In [ ]:
user_preferences_1 = {
    "Restaurant": {
        "weight": 3,
        "sub_category": 4
    },
    "Religion": {
        "weight": 1,
        "sub_category": 4
    },
    "Recreation": {
        "weight": 2,
        "sub_category": 1
    },
    "Cronotipo": 2
}

In [ ]:
filtrado_prueba = filtrar_locales(df_locales_ml, 34.058092, -118.292130, 1, user_preferences_1)

In [ ]:
filtrado_prueba

,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews,religion,recreation,bienestar,avg_rating_correction,forecast,tendencia,rating_3_months,time_category,distance,ml_category
131,"847 S Vermont Ave Parking, 3038W W 8th St, Los...",0x80c2c77897fffa8b:0x5c2904deb5734e19,34.056428,-118.291801,3.0,1,0,0,1,NaN,NaN,NaN,NaN,0.0,200.0,bien
185,"mirae medical clinic, 2675 W Olympic Blvd #202...",0x80c2c783a0bd3031:0x117bf83e39fbd829,34.052670,-118.290634,3.5,10,0,0,1,NaN,NaN,NaN,NaN,0.0,600.0,bien
284,"Wi Spa Customer Parking, 2701W 7th St, Los Ang...",0x80c2c77667d9201b:0x4f286f8fc3438bca,34.060011,-118.283239,3.0,7,0,0,1,NaN,NaN,NaN,NaN,0.0,800.0,bien
317,"Subway, 3323 Wilshire Blvd Ground Floor, Los A...",0x80c2c77c6a3968f1:0xee3b615679d47cdf,34.061971,-118.295066,3.8,78,0,0,0,3.920455,fuerte,Media,3.71,0.0,500.0,res
335,"Orange Tree Cafe, 3500 W 6th St, Los Angeles, ...",0x80c2c77d5c06fd19:0xfa9109c7f6fd173f,34.063140,-118.298600,4.3,18,0,0,0,NaN,NaN,NaN,NaN,0.0,800.0,res
446,"Deli-In Snack Shop, 3456 Wilshire Blvd, Los An...",0x80c2b88280c0b541:0x5375db2c95eb9e88,34.061278,-118.299755,4.2,18,0,0,0,NaN,NaN,NaN,NaN,1.0,800.0,res
468,"Samsung Care Pharmacy, 2717 W Olympic Blvd #10...",0x80c2c783cb749e07:0x5a12f305d0d06f10,34.053090,-118.292409,4.0,8,0,0,1,NaN,NaN,NaN,NaN,0.0,600.0,bien
470,"Los Angeles Drugs, 3030 W Olympic Blvd, Los An...",0x80c2b87fb86cf8d3:0xb029ab8aff7f37f,34.052477,-118.301232,4.8,8,0,0,1,NaN,NaN,NaN,NaN,0.0,1000.0,bien
523,"Felipe De Neve Branch Library, 2820 W 6th St, ...",0x80c2c7714a9a36d5:0x3c2bac0f64c68269,34.063313,-118.283971,4.5,36,0,1,0,4.931034,débil,Alta,5.00,0.0,1000.0,rec
656,"Fire Chicken, 3128 W 8th St, Los Angeles, CA 9...",0x80c2c7235173c87f:0xb57e3339e8b835e7,34.057481,-118.295248,4.6,38,0,0,0,4.705882,débil,Alta,4.76,0.0,300.0,res
